# Data Exploration and Visualization 

In this notebook I use Neural Networks to identify underlying patterns in our book data that our stakeholders can use to impact their businesses.


By: Prince Okpoziakpo

In [35]:
import pandas as pd
import numpy as np

In [36]:
# read the book data into the 'books_df' dataframe
books_df = pd.read_csv('../data/isbn13_results.csv').drop(columns=['Unnamed: 0'])
books_df.shape

(2761, 13)

# Data Cleaning

## Removing null values

In [37]:
# get all the books that have a description
books_df = books_df.loc[~books_df.description.isna()] 
books_df.shape

(2620, 13)

In [38]:
# get all the books that have categories
books_df = books_df.loc[~books_df.main_categories.isna()]
books_df.shape

(2581, 13)

In [39]:
category_counts = books_df.categories.value_counts()
category_counts[category_counts == 1]

['Comedy programs']                         1
['Alternative comic books, strips, etc']    1
['Candy']                                   1
['Cinematography']                          1
['Popular music']                           1
                                           ..
['Data structures (Computer science)']      1
['Counterculture']                          1
['Computer engineers']                      1
['Presidents']                              1
['Coma']                                    1
Name: categories, Length: 406, dtype: int64

# Feature Extraction

## Vectorize each document from the corpus

In [40]:
from sklearn.feature_extraction.text import CountVectorizer

### Fitting the vectorizer and transforming the corpus

In [50]:
# instantiate the CountVectorizer object; `stop_words` parameter makes sure 
# we exclude English stop words
vectorizer = CountVectorizer(stop_words="english")

# extract the 'description' column and convert it into an array 
descriptions = books_df.description.to_numpy()

# fit the vectorizer and transform the corpus
bag_of_words = vectorizer.fit_transform(descriptions)

print(f"Vocabulary size: {vectorizer.vocabulary_.__len__()}")
print(f"Sample features: {vectorizer.get_feature_names_out()[1000:1005]}\n")
print(bag_of_words.toarray(), '\n')

print(bag_of_words.shape)

Vocabulary size: 23128
Sample features: ['altar' 'alter' 'alteration' 'altered' 'altering']

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]] 

(2581, 23128)


# Target Extraction

## Exploring the categories of each book

In [42]:
# determine the number of unique categories that exist in the dataset
all_categories = [] 
for categories_list_string in books_df.main_categories: 
    if type(categories_list_string) == str:
        c = categories_list_string.\
        rstrip().\
        lstrip().\
        replace('[', '').\
        replace(']', '').\
        replace("\'", '').\
        replace("\"", '').\
        split(',')
        all_categories += c

print("Number of unique categories: ", len(set(all_categories)))

Number of unique categories:  559


## Fit and transform the targets into a label indicator matrix

In [43]:
from sklearn.preprocessing import MultiLabelBinarizer

In [48]:
# convert the type of the categories from string into an array of categories
all_categories = books_df.main_categories.apply(
    lambda s: s.\
        rstrip().\
        lstrip().\
        replace('[', '').\
        replace(']', '').\
        replace("\'", '').\
        replace("\"", '').\
        split(',')
).to_numpy()

(2581,)

In [45]:
# instantiate the binarizer object
mlb = MultiLabelBinarizer()

# fit and transform the labels of the targets into a label indicator matrix
targets = mlb.fit_transform(all_categories)

# verfiy that the number of rows matches the number of rows in `books_df`
# verify that the number of columns matches the number of unique categories
targets.shape

(2581, 559)

# Model Training

## Training the model

In [46]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split as tts

In [51]:
# Split the dataset into training and test sets
X_train, X_test, y_train, y_test = tts(bag_of_words, targets, random_state=42, test_size=.2)

In [52]:
clf = DecisionTreeClassifier(criterion="entropy")
clf.fit(X_train, y_train)

DecisionTreeClassifier(criterion='entropy')

## Evaluating model performance

In [55]:
from sklearn.model_selection import cross_val_score

In [56]:
cross_val_score(clf, X_test, y_test, cv=5)

array([0.24038462, 0.29807692, 0.22330097, 0.29126214, 0.26213592])

In [61]:
X_test.toarray() 

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])